# Spherical Hashing ([paper](http://ai2-s2-pdfs.s3.amazonaws.com/d705/b079cde9b247dcca2ab7124d9f81e733bb36.pdf))
- Uses SIFT dataset available at [http://corpus-texmex.irisa.fr/](http://corpus-texmex.irisa.fr/).
- XXX: Add usage instructions...

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import numpy.matlib

In [4]:
def spherical_hash(embeddings, num_bits):
    m, dimen = embeddings.shape
    m_4 = m / 4.0

    centers = random_centers(embeddings, num_bits)
    o1, o2, radii, avg, stddev = compute_statistics(embeddings, centers)

    iter_max = 100
    iterations = 1
    while True:
        # Force computation based on intersection of each pair of hyper-spheres
        forces = np.zeros((num_bits, dimen))
        for i in range(num_bits - 1):
            for j in range(i + 1, num_bits):
                force = 0.5 * (o2[i, j] - m_4) / m_4 * (centers[i, :] - centers[j, :])
                forces[i, :] += force / float(num_bits)
                forces[j, :] -= force / float(num_bits)

        # Apply forces
        centers += forces
        o1, o2, radii, avg, stddev = compute_statistics(embeddings, centers)

        if avg <= 0.1 * m_4 and stddev <= 0.15 * m_4:
            break

        if iterations >= iter_max:
            print('Iterations exeeded %d, iter=%d, avg=%f, stddev=%f' \
                % (iter_max, iterations, avg, stddev))
            return

        iterations += 1

    return centers, radii

def random_centers(embeddings, num_bits):
    m, dimen = embeddings.shape
    centers = np.zeros((num_bits, dimen))
    for i in range(num_bits):
        R = np.random.randint(0, m, m)
        sample = embeddings[R[:5], :]
        sample = np.sum(sample, axis=0) / 5
        centers[i, :] = sample[:]
    return centers

def compute_statistics(embeddings, centers):
    m, dimen = embeddings.shape
    num_bits = centers.shape[0]

    dist = dist_mat_euclid(centers, embeddings)
    sorted_dist = np.sort(dist, axis=1)
    radii = sorted_dist[:, m / 2]

    dist = dist <= np.matlib.repmat(radii, m, 1).T # ?
    dist = dist * 1.0 # bool -> real

    # TODO: Off by +1?
    o1 = np.sum(dist, axis=1)
    o2 = np.matmul(dist, dist.T)
    avg, avg2 = 0, 0
    for i in range(num_bits - 1):
        for j in range(i + 1, num_bits):
            avg += abs(o2[i, j] - m / 4.0)
            avg2 += o2[i, j]

    num_bits_sqrd = (num_bits * (num_bits - 1) / 2.0)
    avg /= num_bits_sqrd
    avg2 /= num_bits_sqrd
    stddev = 0
    for i in range(num_bits - 1):
        for j in range(i + 1, num_bits):
            stddev += (o2[i, j] - avg2) ** 2

    stddev = np.sqrt(stddev / num_bits_sqrd)
    return o1, o2, radii, avg, stddev

def dist_mat_euclid(mat1, mat2):
    """
    Euclidian distances between matrix rows.
    Output e.x. `[dist(a[0, :], b), dist(a[1, :], b), ...]`
    """

    sqrd1 = np.matlib.repmat(np.square(mat1).sum(axis=1), mat2.shape[0], 1)
    sqrd2 = np.matlib.repmat(np.square(mat2).sum(axis=1), mat1.shape[0], 1)
    r = np.matmul(mat1, mat2.T)
    dist = np.sqrt(sqrd1 + sqrd2.T - 2 * r.T)
    return dist.T # TODO: Fix

In [ ]:
centers, radii = spherical_hash(embeddings, signature_length)
dists = dist_mat(embeddings_test, centers)
th = np.matlib.repmat(radii.T, dists.shape[0], 1)

bucket_ids = np.zeros(dists.shape)
bucket_ids[dists <= th] = 1
bucket_ids = bucket_ids.astype(np.uint8).tolist()
bucket_ids = map(lambda bid: int(''.join(map(str, bid)), 2), bucket_ids)
return bucket_ids

embeddings_test_df['bucket_ids'] = spherical_hashing(embeddings_test_df, signature_length)
graph_bucket_stats(embeddings_test_df)
graph_bucket_cos_vs_true_cos(embeddings_test_df, signature_length=signature_length)
display_buckets(embeddings_test_df)